In [12]:
# DENSITY ON SHAPE — runs on your two uploaded files (OBJ mesh + CSV points)

import os
import numpy as np
import pandas as pd
import trimesh
from scipy.ndimage import gaussian_filter

# Try marching cubes
try:
    from skimage.measure import marching_cubes
    HAS_SKIMAGE = True
except Exception:
    HAS_SKIMAGE = False

MESH_PATH = "data/green_monkey/all_structure_files/chr1/spatial_data/overall_shapes/chr1_12hrs_vacv_metaball.obj"
POINTS_CSV = "data/green_monkey/all_structure_files/chr1/12hrs/vacv/structure_12hrs_vacv_gene_info.csv"
OUT_PREFIX = "data/green_monkey/va_testing/chr1_12hrs_vacv"

# ---- Utility funcs --
def make_grid(bounds_min, bounds_max, spacing):
    mins = np.array(bounds_min, float)
    maxs = np.array(bounds_max, float)
    size = np.maximum(1, np.ceil((maxs - mins) / spacing).astype(int))
    xs = np.linspace(mins[0], mins[0] + size[0]*spacing, size[0]+1)[:-1]
    ys = np.linspace(mins[1], mins[1] + size[1]*spacing, size[1]+1)[:-1]
    zs = np.linspace(mins[2], mins[2] + size[2]*spacing, size[2]+1)[:-1]
    return mins, spacing, (xs, ys, zs)

def world_to_grid(points, origin, spacing):
    return (points - origin) / spacing

def trilinear_splat(points_g, shape):
    vol = np.zeros(shape, dtype=np.float32)
    if points_g.size == 0:
        return vol
    valid = np.all((points_g >= 0) & (points_g < (np.array(shape)-1)), axis=1)
    p = points_g[valid]
    if p.size == 0:
        return vol
    i = np.floor(p).astype(int)
    f = p - i
    w000 = (1-f[:,0])*(1-f[:,1])*(1-f[:,2])
    w100 = f[:,0]    *(1-f[:,1])*(1-f[:,2])
    w010 = (1-f[:,0])*f[:,1]    *(1-f[:,2])
    w110 = f[:,0]    *f[:,1]    *(1-f[:,2])
    w001 = (1-f[:,0])*(1-f[:,1])*f[:,2]
    w101 = f[:,0]    *(1-f[:,1])*f[:,2]
    w011 = (1-f[:,0])*f[:,1]    *f[:,2]
    w111 = f[:,0]    *f[:,1]    *f[:,2]
    for (dx,dy,dz,w) in [(0,0,0,w000),(1,0,0,w100),(0,1,0,w010),(1,1,0,w110),
                         (0,0,1,w001),(1,0,1,w101),(0,1,1,w011),(1,1,1,w111)]:
        idx = (i[:,0]+dx, i[:,1]+dy, i[:,2]+dz)
        np.add.at(vol, idx, w.astype(np.float32))
    return vol

def trilinear_sample(vol, p_g):
    if p_g.size == 0:
        return np.array([], dtype=np.float32)
    p = np.clip(p_g, 0, np.array(vol.shape)-1.0001)
    i = np.floor(p).astype(int)
    f = p - i
    def g(dx,dy,dz): return vol[i[:,0]+dx, i[:,1]+dy, i[:,2]+dz]
    c000 = g(0,0,0); c100 = g(1,0,0); c010 = g(0,1,0); c110 = g(1,1,0)
    c001 = g(0,0,1); c101 = g(1,0,1); c011 = g(0,1,1); c111 = g(1,1,1)
    c00 = c000*(1-f[:,0]) + c100*f[:,0]
    c10 = c010*(1-f[:,0]) + c110*f[:,0]
    c01 = c001*(1-f[:,0]) + c101*f[:,0]
    c11 = c011*(1-f[:,0]) + c111*f[:,0]
    c0 = c00*(1-f[:,1]) + c10*f[:,1]
    c1 = c01*(1-f[:,1]) + c11*f[:,1]
    return (c0*(1-f[:,2]) + c1*f[:,2]).astype(np.float32)

def density_volume_from_points(points, spacing=3.0, blur_sigma_vox=2.0, bounds=None):
    pts = np.asarray(points, float)
    if pts.size == 0:
        raise ValueError("No points provided for density.")
    if bounds is None:
        pad = 3*spacing
        mins = pts.min(0) - pad
        maxs = pts.max(0) + pad
    else:
        mins, maxs = bounds
    origin, sp, axes = make_grid(mins, maxs, spacing)
    shape = (len(axes[0]), len(axes[1]), len(axes[2]))
    pg = world_to_grid(pts, origin, sp)
    vol = trilinear_splat(pg, shape)
    if blur_sigma_vox and blur_sigma_vox > 0:
        vol = gaussian_filter(vol, sigma=blur_sigma_vox, mode='nearest')
    if vol.max() > 0:
        vol = vol / vol.max()
    return vol, origin, sp

def sample_density_on_mesh(mesh, vol, origin, spacing):
    v_world = mesh.vertices
    v_g = world_to_grid(v_world, origin, spacing)
    return trilinear_sample(vol, v_g)

def emboss_mesh(mesh, density, scale=0.8):
    normals = mesh.vertex_normals
    disp = (density - density.mean())[:, None] * normals * scale
    v2 = mesh.vertices + disp
    return trimesh.Trimesh(vertices=v2, faces=mesh.faces, process=False)

# ---- Load data ----
mesh = trimesh.load(MESH_PATH, process=False)
# Ensure normals exist
if mesh.vertex_normals is None or len(mesh.vertex_normals) == 0:
    mesh.rezero()
    mesh.fix_normals()

# Points CSV: try common column names
df = pd.read_csv(POINTS_CSV)
candidate_sets = [
    ("middle_x","middle_y","middle_z"),
    ("x","y","z"),
    ("X","Y","Z")
]
cols = None
for cset in candidate_sets:
    if all(c in df.columns for c in cset):
        cols = cset
        break
if cols is None:
    # Heuristic: first 3 numeric columns
    num_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.number)]
    if len(num_cols) >= 3:
        cols = tuple(num_cols[:3])
    else:
        raise ValueError("Could not find coordinate columns (expected middle_x/middle_y/middle_z).")

points = df[list(cols)].dropna().values

# ---- Build density volume & sample on surface ----
vol, origin, spacing = density_volume_from_points(points, spacing=3.0, blur_sigma_vox=2.0)
d_surf = sample_density_on_mesh(mesh, vol, origin, spacing)  # [0,1]

# ---- Colorize + export (PLY for reliable vertex colors) ----
# Map density to 0..255 using a simple gray ramp (renderer can apply colormap later if desired)
# For immediate color in PLY, encode as RGB gradient (here grayscale).
rgb = (np.stack([d_surf, d_surf, d_surf], axis=1) * 255.0).astype(np.uint8)
mesh_color = mesh.copy()
mesh_color.visual.vertex_colors = rgb
mesh_color.export(OUT_PREFIX + "_density_color.ply")

# ---- Emboss and export ----
emb = emboss_mesh(mesh, d_surf, scale=0.8)
emb.visual.vertex_colors = rgb
emb.export(OUT_PREFIX + "_density_embossed.ply")


result_files = {
    "colored_mesh": OUT_PREFIX + "_density_color.ply",
    "embossed_mesh": OUT_PREFIX + "_density_embossed.ply",
}
result_files


{'colored_mesh': 'data/green_monkey/va_testing/chr1_12hrs_vacv_density_color.ply',
 'embossed_mesh': 'data/green_monkey/va_testing/chr1_12hrs_vacv_density_embossed.ply'}

In [13]:
# points_to_shape_with_density.py
# ------------------------------------------------------------
# Inputs (ONLY points):
#   - POINTS_CSV: CSV with columns: middle_x, middle_y, middle_z (or x,y,z)
#
# Outputs (PLY, ready for Three.js PLYLoader):
#   - <OUT_PREFIX>_density_color.ply      (shape mesh, vertex-colored by fine density)
#   - <OUT_PREFIX>_density_embossed.ply   (same mesh, embossed by fine density)
#
# Notes:
#   • Two-scale KDE: coarse sigma -> shape surface; fine sigma -> surface color/emboss.
#   • Shells are computed from fine density, then masked to the shape interior.
#   • Change GRID_MAX_VOX, SIGMA_SHAPE, SIGMA_FINE to suit data scale.

import os
import numpy as np
import pandas as pd
import trimesh
from scipy.ndimage import gaussian_filter

try:
    from skimage.measure import marching_cubes
except Exception as e:
    raise ImportError("scikit-image is required for marching_cubes. pip install scikit-image") from e

# -------------------------- CONFIG --------------------------
POINTS_CSV = "data/green_monkey/all_structure_files/chr1/12hrs/vacv/structure_12hrs_vacv_gene_info.csv"
OUT_PREFIX  = "data/green_monkey/va_testing/chr1_12hrs_vacv_from_points"

# Grid / KDE settings
GRID_MAX_VOX    = 160        # max voxels along the longest axis (80–220 is typical)
MARGIN_RATIO    = 0.05       # padding around the points bbox (5–10%)
SIGMA_SHAPE     = 3.0        # blur (in voxels) for SHAPE extraction (coarser = smoother)
SHAPE_Q         = 0.35       # quantile threshold for SHAPE surface (0.25–0.45 typical)
SIGMA_FINE      = 1.5        # blur (in voxels) for COLOR density (finer = more local variation)
SHELL_QS        = (0.30, 0.60, 0.85)
EMBOSS_SCALE    = 0.8        # world-units displacement multiplier for embossed mesh
PCT_STRETCH     = (5, 95)    # percentile stretch for color contrast

# Column fallbacks
CAND_COLSETS = [
    ("middle_x","middle_y","middle_z"),
    ("x","y","z"),
    ("X","Y","Z")
]

# -------------------------- HELPERS --------------------------
def pick_xyz_columns(df):
    for cols in CAND_COLSETS:
        if all(c in df.columns for c in cols):
            return list(cols)
    num_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.number)]
    if len(num_cols) >= 3:
        return num_cols[:3]
    raise ValueError("Could not find coordinate columns (expected middle_x/middle_y/middle_z).")

def auto_grid(points, max_vox=160, margin_ratio=0.05):
    pts = np.asarray(points, float)
    mins = pts.min(0); maxs = pts.max(0)
    size = maxs - mins
    margin = size.max() * margin_ratio
    mins -= margin; maxs += margin
    size = maxs - mins
    spacing = size.max() / max_vox
    nx, ny, nz = np.maximum(1, np.ceil(size / spacing).astype(int))
    return mins, spacing, (nx, ny, nz)

def trilinear_splat(points_w, origin, spacing, shape):
    vol = np.zeros(shape, dtype=np.float32)
    p = (points_w - origin) / spacing
    valid = np.all((p >= 0) & (p < (np.array(shape) - 1)), axis=1)
    p = p[valid]
    if len(p) == 0: return vol
    i = np.floor(p).astype(int)
    f = p - i
    w000 = (1-f[:,0])*(1-f[:,1])*(1-f[:,2])
    w100 = f[:,0]    *(1-f[:,1])*(1-f[:,2])
    w010 = (1-f[:,0])*f[:,1]    *(1-f[:,2])
    w110 = f[:,0]    *f[:,1]    *(1-f[:,2])
    w001 = (1-f[:,0])*(1-f[:,1])*f[:,2]
    w101 = f[:,0]    *(1-f[:,1])*f[:,2]
    w011 = (1-f[:,0])*f[:,1]    *f[:,2]
    w111 = f[:,0]    *f[:,1]    *f[:,2]
    for (dx,dy,dz,w) in [(0,0,0,w000),(1,0,0,w100),(0,1,0,w010),(1,1,0,w110),
                         (0,0,1,w001),(1,0,1,w101),(0,1,1,w011),(1,1,1,w111)]:
        np.add.at(vol, (i[:,0]+dx, i[:,1]+dy, i[:,2]+dz), w.astype(np.float32))
    return vol

def marching(vol, level, origin, spacing):
    verts, faces, _, _ = marching_cubes(
        volume=vol, level=level, spacing=(spacing, spacing, spacing)
    )
    return trimesh.Trimesh(vertices=verts + origin, faces=faces, process=False)

def sample_volume(vol, points_w, origin, spacing):
    p = (points_w - origin) / spacing
    p = np.clip(p, 0, np.array(vol.shape)-1.0001)
    i = np.floor(p).astype(int)
    f = p - i
    def g(dx,dy,dz): return vol[i[:,0]+dx, i[:,1]+dy, i[:,2]+dz]
    c000 = g(0,0,0); c100 = g(1,0,0); c010 = g(0,1,0); c110 = g(1,1,0)
    c001 = g(0,0,1); c101 = g(1,0,1); c011 = g(0,1,1); c111 = g(1,1,1)
    c00 = c000*(1-f[:,0]) + c100*f[:,0]
    c10 = c010*(1-f[:,0]) + c110*f[:,0]
    c01 = c001*(1-f[:,0]) + c101*f[:,0]
    c11 = c011*(1-f[:,0]) + c111*f[:,0]
    c0 = c00*(1-f[:,1]) + c10*f[:,1]
    c1 = c01*(1-f[:,1]) + c11*f[:,1]
    return (c0*(1-f[:,2]) + c1*f[:,2]).astype(np.float32)

def emboss_mesh(mesh, values, scale=0.8):
    n = mesh.vertex_normals
    disp = (values - values.mean())[:, None] * n * scale
    return trimesh.Trimesh(vertices=mesh.vertices + disp, faces=mesh.faces, process=False)

def density_rgba(values01, cmap='Reds'):
    """
    Map values in [0,1] to RGBA colors.
    - If you have matplotlib installed, use any cmap name (e.g., 'Reds', 'OrRd', 'magma', 'viridis').
    - If matplotlib isn't available, falls back to a built-in red ramp.
    """
    try:
        import matplotlib.cm as cm
        import matplotlib.colors as mcolors
        lut = cm.get_cmap(cmap)
        rgba = lut(np.clip(values01, 0, 1))   # float64 Nx4 in 0..1
        return (rgba * 255).astype('uint8')   # uint8 Nx4
    except Exception:
        # Fallback: simple dark→bright red ramp
        v = np.clip(values01, 0, 1).astype(np.float32)
        r = v
        g = 0.15 * (1.0 - v)
        b = 0.15 * (1.0 - v)
        a = np.ones_like(v)
        return (np.stack([r, g, b, a], 1) * 255).astype('uint8')

# -------------------------- PIPELINE --------------------------
def main():
    # 1) Load points
    os.makedirs(os.path.dirname(OUT_PREFIX), exist_ok=True)
    df = pd.read_csv(POINTS_CSV)
    cols = pick_xyz_columns(df)
    pts = df[cols].dropna().values.astype(np.float32)
    if len(pts) < 8:
        raise ValueError("Not enough points to build a density surface.")

    # 2) Build grid
    origin, spacing, (nx, ny, nz) = auto_grid(pts, max_vox=GRID_MAX_VOX, margin_ratio=MARGIN_RATIO)
    shape = (nx, ny, nz)

    # 3) Two KDE volumes: coarse (shape), fine (color/emboss)
    vol_raw = trilinear_splat(pts, origin, spacing, shape)
    vol_shape = gaussian_filter(vol_raw, sigma=SIGMA_SHAPE, mode='nearest')
    vol_fine  = gaussian_filter(vol_raw, sigma=SIGMA_FINE,  mode='nearest')

    # normalize to [0,1] (avoid divide-by-zero)
    if vol_shape.max() > 0: vol_shape = vol_shape / vol_shape.max()
    if vol_fine.max()  > 0: vol_fine  = vol_fine  / vol_fine.max()

    # 4) Extract shape surface from coarse volume (quantile threshold)
    shape_vals = vol_shape[vol_shape > 0]
    if shape_vals.size == 0:
        raise RuntimeError("Coarse volume is empty; check grid and sigma.")
    level_shape = np.quantile(shape_vals, SHAPE_Q)
    mesh_shape = marching(vol_shape, level_shape, origin, spacing)

    # 5) Mask fine density to the shape interior (keep shells inside)
    inside_mask = (vol_shape >= level_shape).astype(np.float32)
    vol_fine_masked = vol_fine * inside_mask
    if vol_fine_masked.max() > 0:
        vol_fine_masked /= vol_fine_masked.max()

    # # 6) Color the shape by fine density (with percentile stretch for contrast)
    # d_surf = sample_volume(vol_fine_masked, mesh_shape.vertices, origin, spacing)
    # p_lo, p_hi = np.percentile(d_surf, PCT_STRETCH)
    # d01 = np.clip((d_surf - p_lo) / max(1e-6, (p_hi - p_lo)), 0, 1)
    # rgba = density_rgba(d01)
    # mesh_col = mesh_shape.copy()
    # mesh_col.visual.vertex_colors = rgba
    # mesh_col.export(OUT_PREFIX + "_density_color.ply")

    # 6) Color the shape by fine density (with robust contrast + dark colormap)
    d_surf = sample_volume(vol_fine_masked, mesh_shape.vertices, origin, spacing)
    
    # Debug: print distribution so you can see what's going on
    p0, p5, p50, p95, p100 = np.percentile(d_surf, [0, 5, 50, 95, 100])
    print(f"[density on surface] min={p0:.4g}  p5={p5:.4g}  med={p50:.4g}  p95={p95:.4g}  max={p100:.4g}")
    
    # Primary stretch
    p_lo, p_hi = np.percentile(d_surf, PCT_STRETCH)
    rng = p_hi - p_lo
    
    # If the dynamic range collapses, try safer fallbacks
    if rng < 1e-6:
        # 1) fall back to min/max
        p_lo, p_hi = float(d_surf.min()), float(d_surf.max())
        rng = p_hi - p_lo
        print(f"[stretch] collapsed; using min/max: {p_lo:.4g}..{p_hi:.4g}")
    
    if rng < 1e-6:
        # 2) still flat → re-make fine KDE with smaller sigma and resample once
        SIGMA_FINE_ALT = max(0.6, SIGMA_FINE * 0.5)
        print(f"[stretch] still flat; retrying with SIGMA_FINE={SIGMA_FINE_ALT}")
        vol_fine_alt = gaussian_filter(vol_raw, sigma=SIGMA_FINE_ALT, mode='nearest')
        vol_fine_alt = vol_fine_alt * (vol_shape >= level_shape).astype(np.float32)
        # don't renormalize to [0,1] globally; keep relative variation
        d_surf = sample_volume(vol_fine_alt, mesh_shape.vertices, origin, spacing)
        p_lo, p_hi = np.percentile(d_surf, PCT_STRETCH)
        rng = max(1e-6, p_hi - p_lo)
    
    # Final normalized values in [0,1]
    d01 = np.clip((d_surf - p_lo) / rng, 0, 1)
    
    # Use a dark-to-bright sequential map so low values aren't white
    rgba = density_rgba(d01, cmap='magma')   # alternatives: 'inferno', 'turbo', 'Reds_r'
    mesh_col = mesh_shape.copy()
    mesh_col.visual.vertex_colors = rgba
    mesh_col.export(OUT_PREFIX + "_density_color.ply")
    
    # 7) Emboss (unchanged, but now using the same d01 + magma colors)
    mesh_emb = emboss_mesh(mesh_shape, d01, scale=EMBOSS_SCALE)
    mesh_emb.visual.vertex_colors = rgba
    mesh_emb.export(OUT_PREFIX + "_density_embossed.ply")


    # # 7) Emboss the shape by fine density variation and export (also colored)
    # mesh_emb = emboss_mesh(mesh_shape, d01, scale=EMBOSS_SCALE)
    # mesh_emb.visual.vertex_colors = rgba
    # mesh_emb.export(OUT_PREFIX + "_density_embossed.ply")


    print("Done:")
    print("  ", OUT_PREFIX + "_density_color.ply")
    print("  ", OUT_PREFIX + "_density_embossed.ply")

if __name__ == "__main__":
    main()


[density on surface] min=0  p5=0  med=0  p95=4.173e-10  max=1.292e-09
[stretch] collapsed; using min/max: 0..1.292e-09
[stretch] still flat; retrying with SIGMA_FINE=0.75
Done:
   data/green_monkey/va_testing/chr1_12hrs_vacv_from_points_density_color.ply
   data/green_monkey/va_testing/chr1_12hrs_vacv_from_points_density_embossed.ply


/tmp/ipykernel_10283/3932937958.py:125: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  lut = cm.get_cmap(cmap)
